!pip install sklearn

In [66]:
!pip install gensim

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 24.0 MB 57 kB/s             
     |████████████████████████████████| 58 kB 337 kB/s            
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


# Part-of-speech tagging

### Import libraries

In [2]:
import os
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE


### Download dataste

In [3]:
import urllib.request  #  download files
import zipfile  #  unzip files

dataset_folder = os.path.join(os.getcwd(), "OriginalDataset")

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

url = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip'

dataset_path = os.path.join(dataset_folder, "dependency_treebank.zip")

if not os.path.exists(dataset_path):
    urllib.request.urlretrieve(url, dataset_path)
    print("Successful download")

with zipfile.ZipFile(dataset_path,"r") as zip_ref:
    zip_ref.extractall(dataset_folder)
print("Successful extraction")

Successful extraction


### Split dataset

In [4]:
train_path = './Dataset/train/'# Should we use: os.path.join(os.getcwd(), "OriginalDataset")
val_path = './Dataset/val/'
test_path = './Dataset/test/'
original_path = dataset_folder +'/dependency_treebank/'

In [5]:
if not os.path.exists(train_path):
    print("making directory")
    os.makedirs(train_path)
if not os.path.exists(val_path):
    os.makedirs(val_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

In [6]:
original_dataset = os.listdir(original_path)
print(original_dataset)

['wsj_0095.dp', 'wsj_0184.dp', 'wsj_0177.dp', 'wsj_0037.dp', 'wsj_0126.dp', 'wsj_0066.dp', 'wsj_0052.dp', 'wsj_0112.dp', 'wsj_0003.dp', 'wsj_0143.dp', 'wsj_0153.dp', 'wsj_0013.dp', 'wsj_0102.dp', 'wsj_0042.dp', 'wsj_0076.dp', 'wsj_0136.dp', 'wsj_0027.dp', 'wsj_0167.dp', 'wsj_0194.dp', 'wsj_0085.dp', 'wsj_0007.dp', 'wsj_0147.dp', 'wsj_0056.dp', 'wsj_0116.dp', 'wsj_0180.dp', 'wsj_0091.dp', 'wsj_0122.dp', 'wsj_0062.dp', 'wsj_0173.dp', 'wsj_0033.dp', 'wsj_0023.dp', 'wsj_0163.dp', 'wsj_0072.dp', 'wsj_0132.dp', 'wsj_0081.dp', 'wsj_0190.dp', 'wsj_0106.dp', 'wsj_0046.dp', 'wsj_0157.dp', 'wsj_0017.dp', 'wsj_0006.dp', 'wsj_0146.dp', 'wsj_0057.dp', 'wsj_0117.dp', 'wsj_0181.dp', 'wsj_0090.dp', 'wsj_0123.dp', 'wsj_0063.dp', 'wsj_0172.dp', 'wsj_0032.dp', 'wsj_0022.dp', 'wsj_0162.dp', 'wsj_0073.dp', 'wsj_0133.dp', 'wsj_0080.dp', 'wsj_0191.dp', 'wsj_0107.dp', 'wsj_0047.dp', 'wsj_0156.dp', 'wsj_0016.dp', 'wsj_0094.dp', 'wsj_0185.dp', 'wsj_0176.dp', 'wsj_0036.dp', 'wsj_0127.dp', 'wsj_0067.dp', 'wsj_0053

In [7]:
original_dataset.sort()
org_train = original_dataset[0:100]
org_val = original_dataset[100:150]
org_test = original_dataset[150:]

print(org_train)
print(org_val)
print(org_test)


['wsj_0001.dp', 'wsj_0002.dp', 'wsj_0003.dp', 'wsj_0004.dp', 'wsj_0005.dp', 'wsj_0006.dp', 'wsj_0007.dp', 'wsj_0008.dp', 'wsj_0009.dp', 'wsj_0010.dp', 'wsj_0011.dp', 'wsj_0012.dp', 'wsj_0013.dp', 'wsj_0014.dp', 'wsj_0015.dp', 'wsj_0016.dp', 'wsj_0017.dp', 'wsj_0018.dp', 'wsj_0019.dp', 'wsj_0020.dp', 'wsj_0021.dp', 'wsj_0022.dp', 'wsj_0023.dp', 'wsj_0024.dp', 'wsj_0025.dp', 'wsj_0026.dp', 'wsj_0027.dp', 'wsj_0028.dp', 'wsj_0029.dp', 'wsj_0030.dp', 'wsj_0031.dp', 'wsj_0032.dp', 'wsj_0033.dp', 'wsj_0034.dp', 'wsj_0035.dp', 'wsj_0036.dp', 'wsj_0037.dp', 'wsj_0038.dp', 'wsj_0039.dp', 'wsj_0040.dp', 'wsj_0041.dp', 'wsj_0042.dp', 'wsj_0043.dp', 'wsj_0044.dp', 'wsj_0045.dp', 'wsj_0046.dp', 'wsj_0047.dp', 'wsj_0048.dp', 'wsj_0049.dp', 'wsj_0050.dp', 'wsj_0051.dp', 'wsj_0052.dp', 'wsj_0053.dp', 'wsj_0054.dp', 'wsj_0055.dp', 'wsj_0056.dp', 'wsj_0057.dp', 'wsj_0058.dp', 'wsj_0059.dp', 'wsj_0060.dp', 'wsj_0061.dp', 'wsj_0062.dp', 'wsj_0063.dp', 'wsj_0064.dp', 'wsj_0065.dp', 'wsj_0066.dp', 'wsj_0067

In [8]:
for f in org_train:
    os.rename(original_path+f, train_path+f)
for f in org_val:
    os.rename(original_path+f, val_path+f)
for f in org_test:
    os.rename(original_path+f, test_path+f)


### Preprocessing

In [9]:
def lower(lst): # DO WE NEED THIS?
    lowercase_list = [x.lower() for x in lst]
    return lowercase_list

### Create dataframe

In [10]:
def create_dataframe(dataset_path: str):
    dataframe_rows = []
    documents = os.listdir(dataset_path)
    for document in documents:
        path = os.path.join(dataset_path, document)
        with open(path, 'r') as f:
            a = [[x for x in ln.split()] for ln in f]
        a2 = [x for x in a if x != []] # OBS! Removing all empty lines in file so we can make an array
        arr = np.array(a2)
        text = arr[:, 0]
        tagg = arr[:, 1]
        POStuple = [(text[x], tagg[x]) for x in range(len(text))]
        document_id = int(document[4:8])
        dataframe_row = {
            "document_id": document_id,
            "text": text,
            "POS": POStuple
        }
        dataframe_rows.append(dataframe_row)
    # transform the list of rows in a proper dataframe
    df = pd.DataFrame(dataframe_rows)
    df = df[["document_id", "text", "POS"]]

    return df

In [11]:
df_train = create_dataframe(train_path)
df_val = create_dataframe(val_path)
df_test = create_dataframe(test_path)


### Embed the words using GloVe embedding

embeddings_dict = {}

#### Load glove model

In [12]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(embedding_dimension: int = 50) -> gensim.models.keyedvectors.KeyedVectors:
    download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
    # Check download
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Word2Vec: 300")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model

In [18]:
embedding_dimension = 50

embedding_model = load_embedding_model(embedding_dimension)

#### Get vocabulary

In [14]:
def get_word_list(data_frame):
    big_list = []
    text = data_frame["text"]
    for row in data_frame["text"]:
        big_list += row.tolist()

    word_list = set(big_list)
    return word_list

In [15]:
word_list_train = get_word_list(df_train)
word_list_val = get_word_list(df_val)
word_list_test = get_word_list(df_test)

#### Find OOV

In [16]:
# Function definition

def check_OOV_terms(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                    word_list):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_listing: dataset specific vocabulary (list)

    :return
        - list of OOV terms
    """
    embedding_vocabulary = set(embedding_model.index_to_key) # Was previously: set(embedding_model.vocab.keys())
    print(len(embedding_vocabulary))
    oov = set(word_list).difference(embedding_vocabulary)
    return list(oov)


In [63]:
oov1_terms = check_OOV_terms(embedding_model, word_list_train)
#--------------------------OBS--------------------------------------------#
# To make sure OOV2 does not contain words from OOV1 a check is 
# implemented during the embedding in the word to index function. 
# It might be an idea to include this check in the chekc_OOV_terms as well. 
# Question for the professor?
#--------------------------OBS--------------------------------------------#
oov2_terms = check_OOV_terms(embedding_model, word_list_val)
oov3_terms = check_OOV_terms(embedding_model, word_list_test)


print("Total OOV terms: {0} ({1:.2f}%)".format(len(oov_terms), float(len(oov_terms)) / len(word_list_train)))

400000
400000
400000
Total OOV terms: 2346 (0.29%)


### Create word_to_index (for one-hot encoding)

In [57]:
from collections import OrderedDict

def create_word_to_idx(vocabulary):
    word_to_idx = OrderedDict()
    
    current_idx = 0
    for word in vocabulary:
        if word not in word_to_idx:
            word_to_idx[word] = current_idx
            current_idx += 1
            
    return word_to_idx

def update_word_to_idx(old_word_to_idx, new_words):
    word_to_idx = old_word_to_idx.copy()
    current_idx = len(word_to_idx)
    for word in new_words:
        if word not in word_to_idx:
            word_to_idx[word] = current_idx
            current_idx += 1
            
    return word_to_idx
    

In [64]:
word_to_idx_v1 = create_word_to_idx(set(embedding_model.index_to_key))
word_to_idx_v2 = update_word_to_idx(word_to_idx_v1, oov1_terms)
word_to_idx_v3 = update_word_to_idx(word_to_idx_v2, oov2_terms)

"""----------------------OBS------------------------------------
Should we also compute word to index for the test set and embed
these words, and then use all of them for training and validation
or should we keep the test set seperate and only compute this for
testing?
----------------------OBS------------------------------------"""

'----------------------OBS------------------------------------\nShould we also compute word to index for the test set and embed\nthese words, and then use all of them for training and validation\nor should we keep the test set seperate and only compute this for\ntesting?\n----------------------OBS------------------------------------'

### Create embedding matrix

In [58]:
# OBS Computes the OOV with random embeddings
def create_embedding_matrix(embedding_model, embedding_dimension, word_to_idx):
    embedding_matrix = np.zeros((len(word_to_idx), embedding_dimension), dtype=np.float32)
    for word, idx in word_to_idx.items():
        try:
            embedding_vector = embedding_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector
                                
    return embedding_matrix

def expand_embedding_matrix(embedding_model, embedding_dimension, word_to_idx, old_embedding_matrix):
    embedding_matrix = np.zeros((len(word_to_idx), embedding_dimension), dtype=np.float32)
    embedding_matrix[0:len(old_embedding_matrix)] = old_embedding_matrix
    
    for word, idx in word_to_idx.items():
        if idx >= len(old_embedding_matrix):
            try:
                embedding_vector = embedding_model[word]
            except (KeyError, TypeError):
                embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)
                
            embedding_matrix[idx] = embedding_vector
    
    return embedding_matrix                                

In [65]:
embedding_matrix_v1 = create_embedding_matrix(embedding_model, embedding_dimension, word_to_idx_v1)
embedding_matrix_v2 = expand_embedding_matrix(embedding_model, embedding_dimension, word_to_idx_v2, embedding_matrix_v1)
embedding_matrix_v3 = expand_embedding_matrix(embedding_model, embedding_dimension, word_to_idx_v3, embedding_matrix_v2)

print(embedding_matrix_v1.shape)
print(embedding_matrix_v2.shape)
print(embedding_matrix_v3.shape)


(400000, 50)
(402346, 50)
(403290, 50)


#### Test (Remove later)


In [61]:
vocab = ['Hello', 'Solveig', 'is', 'very', 'wierd', '.']

w_t_idx = create_word_to_idx(vocab)
print(w_t_idx)

embed_matrix = create_embedding_matrix(None, 5, w_t_idx)
print(embed_matrix)

new_words = ['Andrea', 'is', 'super', 'cool', '.']
new_w_t_idx = update_word_to_idx(w_t_idx, new_words)
print(new_w_t_idx)

new_embed_matrix = expand_embedding_matrix(None, 5, new_w_t_idx, embed_matrix)
print(new_embed_matrix)

OrderedDict([('Hello', 0), ('Solveig', 1), ('is', 2), ('very', 3), ('wierd', 4), ('.', 5)])
[[ 0.02183098  0.02402802  0.03118286  0.04769417  0.01872175]
 [-0.01728713  0.02662692  0.02155588 -0.00091611 -0.01262367]
 [-0.00816416  0.03744153 -0.00933892  0.03142535 -0.04727874]
 [ 0.01423092  0.00143279 -0.02720169 -0.00628638  0.03455986]
 [-0.02372587  0.01482276 -0.0354635   0.04404848 -0.01387006]
 [ 0.03148971  0.03084763 -0.00861655  0.00823898 -0.00565485]]
OrderedDict([('Hello', 0), ('Solveig', 1), ('is', 2), ('very', 3), ('wierd', 4), ('.', 5), ('Andrea', 6), ('super', 7), ('cool', 8)])
[[ 0.02183098  0.02402802  0.03118286  0.04769417  0.01872175]
 [-0.01728713  0.02662692  0.02155588 -0.00091611 -0.01262367]
 [-0.00816416  0.03744153 -0.00933892  0.03142535 -0.04727874]
 [ 0.01423092  0.00143279 -0.02720169 -0.00628638  0.03455986]
 [-0.02372587  0.01482276 -0.0354635   0.04404848 -0.01387006]
 [ 0.03148971  0.03084763 -0.00861655  0.00823898 -0.00565485]
 [-0.02011299  0.